# Information Visualization Course Project

**Authors:** Anastasiia Marchenko and Silje Eriksen, exchange students, course «Information Visualization», University of Bologna

**Title:** Green Practices: Identification of Current Research Trends in the Academic Literature on Environmental Sociology

**Key research question:** What are the main trends in the study of environmental practices over the last 10 years (from 2013 to 2022 inclusive) in the field of environmental sociology?

**Research sub-questions:**
1. What are the conceptual trends in environmental practices research for 2013–2022?
2. What are the methodological trends in environmental practices research for 2013–2022?
3. Is there (what is) the relationship between the types of publications (conceptual and methodological) on environmental practices and the level of academic reputation (expressed by quartile) in the various journals that publish them for 2013–2022?
4. What are the geographical features in environmental practice research for 2013-2022? Which authors from which countries contribute the most to this study and is there the relationship between the geographical origin of the authors and the prevailing topics covered in their publications?

---

## Table of contents:
- [Chapter 0. Preparation for work](#Chapter_0_Preparation_for_work)
- [Chapter 1. Data collection from Scopus](#Chapter_1_Data_collection_from_Scopus)
- [Chapter 2. Data collection from Scimago Journal Rank (SJR)](#Chapter_2_Data_collection_from_Scimago_Journal_Rank_(SJR))
- [Chapter 3. Data integration from Scopus and Scimago Journal Rank (SJR)](#Chapter_3_Data_integration_from_Scopus_and_Scimago_Journal_Rank_(SJR))
- [Chapter 4. Preliminary examination of the database and preparation for analysis](#Chapter_4_Preliminary_examination_of_the_database_and_preparation_for_analysis)
    - [4.1. Analysis of variables' types, missing values and duplicates](#4_1_Analysis_of_variables'_types,_missing_values_and_duplicates)
    - [4.2. ](#)
    - [4.3. ](#)
    - [4.4. ](#)
- [Chapter 5. ](#Chapter_5)
- [Chapter 6. Main conclusions and limitations](#Chapter_6_Main_conclusions_on_the_results_obtained_and_the_limitations_of_the_project)

## Some useful links:
- Scopus [[Source](https://www.scopus.com/)]
- Scopus API (Application Programming Interface) Documentation [[Source 1](https://dev.elsevier.com/technical_documentation.html); [Source 2](https://dev.elsevier.com/api_docs.html)]
- The "pybliometrics" package in the Python programming language [[Source 1](https://github.com/pybliometrics-dev/pybliometrics); [Source 2](https://pybliometrics.readthedocs.io/en/stable/index.html)]
- Scimago Journal Rank (SJR) [[Source](https://www.scimagojr.com/)]

---
## Chapter_0_Preparation_for_work

In [1]:
## 0 ## Installing and importing modules/libraries
#!pip install pandas # To work with dataframes
#!pip install pybliometrics # To work with api.elsevier.com
#!pip install requests # To work with all kinds of HTTP requests
#!pip install os # To work with the operating system


import pandas
from pybliometrics.scopus import ScopusSearch
#from pybliometrics.scopus import AbstractRetrieval
import requests
import os


# Specifying the API key for successful package importing
# "a6b49bc00cce366026d4cfd9396ac572" - At the moment (28/01/24), the quota limit HAS NOT been spent
# "c4b35f1579a33db64d94f97c723a60d8" - At the moment (28/01/24), the quota limit HAS been spent


#help()

---
## Chapter_1_Data_collection_from_Scopus

In [2]:
## 1 ## Request formation with all the necessary parameters

# Specifying the API key
api_key = "a6b49bc00cce366026d4cfd9396ac572"

# Specifying the query
query = '( TITLE-ABS-KEY ( "green practice*" OR "environment* practice*" OR "ecolog* practice*" OR "eco-practice*" OR "green behav*" OR "environment* behav*" OR "ecolog* behav*" OR "eco-behav*" ) AND PUBYEAR > 2012 AND PUBYEAR < 2023 ) AND ( sociology ) AND DOCTYPE("ar") AND SUBJAREA("SOCI") AND LANGUAGE("English")'

The query string written above is provided in a slightly different form, since all fields except “LIMIT-TO()” work for the class being used.

**The formula of the search query from the web version of Scopus:** ( TITLE-ABS-KEY ( "green practice*" OR "environment* practice*" OR "ecolog* practice*" OR "eco-practice*" OR "green behav*" OR "environment* behav*" OR "ecolog* behav*" OR "eco-behav*" ) AND PUBYEAR > 2012 AND PUBYEAR < 2023 ) AND ( sociology ) AND ( LIMIT-TO ( DOCTYPE , "ar" ) ) AND ( LIMIT-TO ( SUBJAREA , "SOCI" ) ) AND ( LIMIT-TO ( LANGUAGE , "English" ) )

**Explanation for the query parameters used (for more information, see [here](https://www.scopus.com/search/form.uri?display=advanced)):**
1. Field codes:
    - TITLE-ABS-KEY - A combined field that searches abstracts, keywords, and document titles.
    - PUBYEAR - A numeric field indicating the year of publication.
    - DOCTYPE - Limits your search to document types - article (ar), review (re), book chapter(ch), etc.
    - LANGUAGE - The language in which the original document was written.
    - SUBJAREA - A search field which returns documents related to a specific field of science.
2. Operators:
    - AND - Use AND when you want your results to include all terms and the terms may be far apart.
    - OR - Use OR when your results must include one or more of the terms (such as synonyms, alternate spellings, or abbreviations). Documents that contain any of the words will be found.
3. Wildcards:
    - Asterisk (*) - Replace multiple characters anywhere in a word. The asterisk replaces 0 or more characters, so it can be used to find any number or to indicate a character that may or may not be present.

In [3]:
## 2 ## Executing the request and saving all the collected data on request to the "response" object
response = ScopusSearch(api_key = api_key
                        , query = query
                        , view = "STANDARD"
                        , verbose = True
                        , subscriber = False)

In [4]:
## 3 ## Determining the number of publications found on request
response.get_results_size()

610

In [5]:
## 4 ## Creating a dataframe in which all information about the collected publications will be saved
all_publications = pandas.DataFrame(response.results)
all_publications

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85150945516,10.1007/s43621-022-00114-6,None,None,How can Quality of Life be Achieved in a Susta...,ar,Article,Wiesli T.X.,None,University of Bern,...,None,None,None,0,1,repositoryvor,Green,None,None,None
1,2-s2.0-85149110210,10.18280/ijsdp.170828,None,None,A Study on the Contribution of Saudi Citizens ...,ar,Article,Khan U.,None,Prince Sattam Bin Abdulaziz University,...,2593-2600,None,None,0,1,publisherfree2read,Bronze,None,None,None
2,2-s2.0-85144925899,10.3390/su142416628,None,None,The Process of Eco-Anxiety and Ecological Grie...,ar,Article,Pihkala P.,None,Helsingin Yliopisto,...,None,None,None,9,1,repositoryvor,Green,None,None,None
3,2-s2.0-85144897738,10.3390/su142416366,None,None,Older Persons’ Perceptions concerning Climate ...,ar,Article,Ayalon L.,None,Bar-Ilan University,...,None,None,None,2,1,repositoryvor,Green,None,None,None
4,2-s2.0-85143796079,10.3390/su142316111,None,None,Rural–Urban Divide: Generation Z and Pro-Envir...,ar,Article,Dąbrowski L.S.,None,Uniwersytet Mikołaja Kopernika w Toruniu,...,None,None,None,4,1,publisherfullgold,Gold,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2-s2.0-84884678671,10.1177/0162243913495924,None,None,Unheeded Science: Taking Precaution out of Tox...,ar,Article,Hoffman K.,None,Universidad de Puerto Rico,...,829-850,None,None,6,0,None,None,None,None,None
606,2-s2.0-84884519014,10.1108/JEA-04-2012-0049,None,None,The relationship between transformational lead...,ar,Article,Keung E.K.,None,None,...,836-854,None,None,41,0,None,None,None,None,None
607,2-s2.0-84879053585,10.1080/13504622.2012.695013,None,None,Use of self-determination theory to support ba...,ar,Article,Karaarslan G.,None,Aǧrı İbrahim Çeçen Üniversitesi;Middle East Te...,...,342-369,None,None,10,0,None,None,None,None,None
608,2-s2.0-84874506315,10.1177/0162243912470726,None,None,Justice as Measure of Nongovernmental Organiza...,ar,Article,Allen B.,None,Virginia Polytechnic Institute and State Unive...,...,224-249,None,None,14,0,None,None,None,None,None


In [6]:
## 5 ## Just in case, saving the database in its original form in an Excel file format called "All_publications_Scopus.xlsx"
all_publications.index = range(1, len(all_publications) + 1)
#all_publications.to_excel("All_publications_Scopus.xlsx")

In [7]:
## 6 ## Additional collection of abstracts, keywords and all authors for the found publications
#for index, row in all_publications.iterrows():
    #scopus_id = row["eid"]
    #try:
        #publication_info = AbstractRetrieval(scopus_id, view = "FULL")
        #all_publications.at[index, "Abstract"] = publication_info.abstract
        #all_publications.at[index, "Keywords"] = publication_info.keywords
        #all_publications.at[index, "Author(s)"] = publication_info.authors
    #except Exception as e:
        #print(f"Error retrieving information for Scopus ID {scopus_id}: {str(e)}")

#all_publications

The code in chunk 6 will not work, as full authorization is required, which we, as students of the University of Bologna, do not have. However, we have a subscription from the university to the Scopus database, so we downloaded the rest of the necessary data manually in a ready-made file format. It was important for us to try to collect data in an automated way, so we decided to collect all the open available data through the API, and get the rest manually, since they cannot be collected automatically. Next, we will upload an additional file and combine it with the already collected database.

We also manually divided the publications into "conceptual", "methodological" and "both" types ones, and added a "Direction" column with this characteristic to this additional file.

**Explanation for the direction types used:**
1. Conceptual - the main purpose of the article is to consider or develop a theory or a new concept.
2. Methodological - the main purpose of the article is to consider or develop an approach to data collection or analysis or a new method.
3. Both - the article combines both of these goals.

In [8]:
## 7 ## Uploading a file with additional collected data (abstracts, keywords and all authors)
additional_data = pandas.read_excel("Abstracts_Keywords_Authors_Scopus.xlsx", index_col = 0)
additional_data

,Authors,Author full names,Author(s) ID,DOI,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,References,Correspondence Address,ISSN,ISBN,CODEN,EID,Direction
0,Rasiah R.; Kaur H.; Baharom A.H.; Turner J.J.;...,"Rasiah, R. (57191888456); Kaur, H. (5721174346...",57191888456; 57211743461; 57217223819; 5720855...,10.12738/jestp.2022.1.0010,https://www.scopus.com/inward/record.uri?eid=2...,"Saito University College, Malaysia; Taylor's U...","Rasiah R., Saito University College, Malaysia;...",Malaysia established its Eleventh Malaysia Pla...,Human Capital; Parental Influence; Perceived S...,"Ahmed S., Reading habits and attitudes of UMSK...","H. Kaur; Taylor's University, Malaysia; email:...",26305984,NaN,NaN,2-s2.0-85128988067,conceptual
1,Zhidebekkyzy A.; Moldabekova A.; Amangeldiyeva...,"Zhidebekkyzy, Aknur (57192831004); Moldabekova...",57192831004; 57207841308; 57918883300; 5720111...,10.14254/2071-789X.2022/15-3/12,https://www.scopus.com/inward/record.uri?eid=2...,"Al-Farabi Kazakh National University, Almaty, ...","Zhidebekkyzy A., Al-Farabi Kazakh National Uni...",Circular economy is one of the best alternativ...,circular economy; pro-environmental behavior; ...,"Ajzen I., The theory of planned behavior, Orga...",NaN,2071789X,NaN,NaN,2-s2.0-85139390951,conceptual
2,Wang H.,"Wang, Huan (57959877600)",57959877600,10.3390/su142114538,https://www.scopus.com/inward/record.uri?eid=2...,"School of Government and Public Affairs, Commu...","Wang H., School of Government and Public Affai...",It is not that the public is unwilling to part...,environmental knowledge; environmental respons...,"Parrado S., Van Ryzin G.G., Bovaird T., Loffle...",H. Wang; School of Government and Public Affai...,20711050,NaN,NaN,2-s2.0-85148103163,conceptual
3,Trendafilova S.; Ziakas V.,"Trendafilova, Sylvia (39763220600); Ziakas, Va...",39763220600; 36563184100,10.3389/fspor.2022.937765,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Kinesiology, Recreation and Spor...","Trendafilova S., Department of Kinesiology, Re...",This paper focuses on the social features of p...,culture; environment; outdoor sports; sport ec...,"Anderson J., Relational places: the surfed wav...","S. Trendafilova; Department of Kinesiology, Re...",26249367,NaN,NaN,2-s2.0-85137731066,conceptual
4,Yazar M.; Hermwille L.; Haarstad H.,"Yazar, Mahir (57201992566); Hermwille, Lukas (...",57201992566; 56582265600; 16052709500,10.1016/j.regsus.2022.11.001,https://www.scopus.com/inward/record.uri?eid=2...,"Centre for Climate and Energy Transformation, ...","Yazar M., Centre for Climate and Energy Transf...",Research on environmental behaviour is often o...,Climate mitigation policies; Decarbonization a...,"Baigorrotegui G., Destabilization of energy re...",M. Yazar; Centre for Climate and Energy Transf...,20970129,NaN,NaN,2-s2.0-85142458078,conceptual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,Curtis D.J.; Howden M.; Curtis F.; McColm I.; ...,"Curtis, David J. (35274373500); Howden, Mark (...",35274373500; 7004004338; 56198896700; 56198640...,10.1017/aee.2014.5,https://www.scopus.com/inward/record.uri?eid=2...,"Institute of Rural Futures, University of New ...","Curtis D.J., Institute of Rural Futures, Unive...",Engaging and exciting students about the envir...,drama; environmental attitudes; environmental ...,"Adcock L., Ballantyne R., Drama as a tool in i...","D.J. Curtis; Institute of Rural Futures, Unive...",08140626,NaN,NaN,2-s2.0-84902076594,conceptual
606,Hoffman K.,"Hoffman, Karen (55865763600)",55865763600,10.1177/0162243913495924,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Sociology and Anthropology, Univ...","Hoffman K., Department of Sociology and Anthro...","In the early 1970s, the idea of precaution-of ...",engagement; environmental practices; governanc...,"Boehmer-Christiansen S., Interpreting the Prec...",K. Hoffman; Department of Sociology and Anthro...,01622439,NaN,NaN,2-s2.0-84884678671,

In [9]:
## 8 ## Concatenating two dataframes by the EID of each publication
all_publications_new = pandas.merge(all_publications
                                    , additional_data
                                    , left_on = "eid"
                                    , right_on = "EID"
                                    , how = "left")

all_publications_new

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,Authors with affiliations,Abstract,Author Keywords,References,Correspondence Address,ISSN,ISBN,CODEN,EID,Direction
0,2-s2.0-85150945516,10.1007/s43621-022-00114-6,None,None,How can Quality of Life be Achieved in a Susta...,ar,Article,Wiesli T.X.,None,University of Bern,...,"Wiesli T.X., Centre for Development and Enviro...",Rural regions in Europe are often structurally...,Europe; Quality of life; Regional development;...,"Felce D., Perry J., Quality of life: its defin...",T.X. Wiesli; Centre for Development and Enviro...,26629984,NaN,NaN,2-s2.0-85150945516,conceptual
1,2-s2.0-85149110210,10.18280/ijsdp.170828,None,None,A Study on the Contribution of Saudi Citizens ...,ar,Article,Khan U.,None,Prince Sattam Bin Abdulaziz University,...,"Khan U., Department of Finance, College of Bus...",This research measures Saudis' environmental a...,awareness; environmental sustainability; pro-e...,"Mao J.G., Teaching environmental awareness in ...","U. Khan; Department of Finance, College of Bus...",17437601,NaN,NaN,2-s2.0-85149110210,conceptual
2,2-s2.0-85144925899,10.3390/su142416628,None,None,The Process of Eco-Anxiety and Ecological Grie...,ar,Article,Pihkala P.,None,Helsingin Yliopisto,...,"Pihkala P., Faculty of Theology, HELSUS Sustai...","As the ecological crisis grows more intense, p...",bereavement; climate anxiety; climate grief; D...,"Steffen W., Richardson K., Rockstrom J., Corne...","P. Pihkala; Faculty of Theology, HELSUS Sustai...",20711050,NaN,NaN,2-s2.0-85144925899,methodological
3,2-s2.0-85144897738,10.3390/su142416366,None,None,Older Persons’ Perceptions concerning Climate ...,ar,Article,Ayalon L.,None,Bar-Ilan University,...,"Ayalon L., Louis and Gabi Weisfeld School of S...",Older people are under-represented in the clim...,activism; barriers; climate change; facilitato...,"Aida J., Hikichi H., Matsuyama Y., Sato Y., Ts...",L. Ayalon; Louis and Gabi Weisfeld School of S...,20711050,NaN,NaN,2-s2.0-85144897738,conceptual
4,2-s2.0-85143796079,10.3390/su142316111,None,None,Rural–Urban Divide: Generation Z and Pro-Envir...,ar,Article,Dąbrowski L.S.,None,Uniwersytet Mikołaja Kopernika w Toruniu,...,"Dąbrowski L.S., Interdisciplinary Doctoral Sch...",Generation Z is the crucial cohort that will h...,Generation Z; pro-environmental behavior; rura...,"Zaccai E., Adams W.M., How far are biodiversit...",L.S. Dąbrowski; Interdisciplinary Doctoral Sch...,20711050,NaN,NaN,2-s2.0-85143796079,conceptual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2-s2.0-84884678671,10.1177/0162243913495924,None,None,Unheeded Science: Taking Precaution out of Tox...,ar,Article,Hoffman K.,None,Universidad de Puerto Rico,...,"Hoffman K., Department of Sociology and Anthro...","In the early 1970s, the idea of precaution-of ...",engagement; environmental practices; governanc...,"Boehmer-Christiansen S., Interpreting the Prec...",K. Hoffman; Department of Sociology and Anthro...,01622439,NaN,NaN,2-s2.0-84884678671,conceptual
606,2-s2.0-84884519014,10.1108/JEA-04-2012-0049,None,None,The relationship between transformational lead...,ar,Article,Keung E.K.,None,None,...,"Keung E.K.; Rockinson-Szapkiw A.J., School of ...",Purpose: The purpose of this study is to exami...,Cultural intelligence; Intercultural schools; ...,"Alon I., Higgins J.M., Global leadership succe...","E. K. Keung; not available, Hong Kong, China; ...",09578234,NaN,NaN,2-s2.0-84884519014,conceptual
607,2-s2.0-84879053585,10.1080/13504622.2012.695013,None,None,Use of self-determination theory to support ba...,ar,Article,Karaarslan G.,None,Aǧrı İbrahim Çeçen Üniversitesi;Middle East Te...,...,"Karaarslan G., Department of Elementary Educat...","In this paper, we examine how the basic psycho...",basic psychological needs; environmental educa...,"Benabou R., Tirole J., Intrinsic motivation an...",G. Karaarslan; Department of Elementary Educat...,13504622,NaN,NaN,2-s2.

In [10]:
## 9 ## Selecting necessary characteristics of publications
characteristics = ["eid"
                   , "doi"
                   , "subtypeDescription"
                   , "affiliation_country"
                   , "coverDate"
                   , "publicationName"
                   , "aggregationType"
                   , "volume"
                   , "issueIdentifier"
                   , "pageRange"
                   , "citedby_count"
                   , "openaccess"
                   , "freetoreadLabel"
                   , "Authors"
                   , "Author full names"
                   , "Author(s) ID"
                   , "Affiliations"
                   , "Authors with affiliations"
                   , "Title"
                   , "Abstract"
                   , "Author Keywords"
                   , "References"
                   , "ISSN"
                   , "Link"
                   , "Direction"]

all_publications_fin = pandas.DataFrame()

for i in all_publications_new:
    if i in characteristics:
        all_publications_fin[i] = all_publications_new[i]

all_publications_fin.index = range(1, len(all_publications_fin) + 1)
all_publications_fin

,eid,doi,subtypeDescription,affiliation_country,coverDate,publicationName,aggregationType,volume,issueIdentifier,pageRange,...,Author full names,Author(s) ID,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,References,ISSN,Direction
1,2-s2.0-85150945516,10.1007/s43621-022-00114-6,Article,Switzerland,2022-12-01,Discover Sustainability,Journal,3,1,None,...,"Wiesli, Thea Xenia (57227811700); Hammer, Thom...",57227811700; 57205888702,https://www.scopus.com/inward/record.uri?eid=2...,"Centre for Development and Environment CDE, Un...","Wiesli T.X., Centre for Development and Enviro...",Rural regions in Europe are often structurally...,Europe; Quality of life; Regional development;...,"Felce D., Perry J., Quality of life: its defin...",26629984,conceptual
2,2-s2.0-85149110210,10.18280/ijsdp.170828,Article,Saudi Arabia,2022-12-01,International Journal of Sustainable Developme...,Journal,17,8,2593-2600,...,"Khan, Uzma (57191380280)",57191380280,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Finance, College of Business Adm...","Khan U., Department of Finance, College of Bus...",This research measures Saudis' environmental a...,awareness; environmental sustainability; pro-e...,"Mao J.G., Teaching environmental awareness in ...",17437601,conceptual
3,2-s2.0-85144925899,10.3390/su142416628,Article,Finland,2022-12-01,Sustainability (Switzerland),Journal,14,24,None,...,"Pihkala, Panu (55734890700)",55734890700,https://www.scopus.com/inward/record.uri?eid=2...,"Faculty of Theology, HELSUS Sustainability Sci...","Pihkala P., Faculty of Theology, HELSUS Sustai...","As the ecological crisis grows more intense, p...",bereavement; climate anxiety; climate grief; D...,"Steffen W., Richardson K., Rockstrom J., Corne...",20711050,methodological
4,2-s2.0-85144897738,10.3390/su142416366,Article,Israel,2022-12-01,Sustainability (Switzerland),Journal,14,24,None,...,"Ayalon, Liat (57368650600); Ulitsa, Natalie (5...",57368650600; 57192891586; 57202607980; 5803295...,https://www.scopus.com/inward/record.uri?eid=2...,"Louis and Gabi Weisfeld School of Social Work,...","Ayalon L., Louis and Gabi Weisfeld School of S...",Older people are under-represented in the clim...,activism; barriers; climate change; facilitato...,"Aida J., Hikichi H., Matsuyama Y., Sato Y., Ts...",20711050,conceptual
5,2-s2.0-85143796079,10.3390/su142316111,Article,Poland,2022-12-01,Sustainability (Switzerland),Journal,14,23,None,...,"Dąbrowski, Leszek S. (57203433553); Środa-Mura...",57203433553; 35362926900; 57209570484; 3533892...,https://www.scopus.com/inward/record.uri?eid=2...,Interdisciplinary Doctoral School of Social Sc...,"Dąbrowski L.S., Interdisciplinary Doctoral Sch...",Generation Z is the crucial cohort that will h...,Generation Z; pro-environmental behavior; rura...,"Zaccai E., Adams W.M., How far are biodiversit...",20711050,conceptual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2-s2.0-84884678671,10.1177/0162243913495924,Article,Puerto Rico,2013-01-01,Science Technology and Human Values,Journal,38,6,829-850,...,"Hoffman, Karen (55865763600)",55865763600,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Sociology and Anthropology, Univ...","Hoffman K., Department of Sociology and Anthro...","In the early 1970s, the idea of precaution-of ...",engagement; environmental practices; governanc...,"Boehmer-Christiansen S., Interpreting the Prec...",01622439,conceptual
607,2-s2.0-84884519014,10.1108/JEA-04-2012-0049,Article,None,2013-01-01,Journal of Educational Administration,Journal,51,6,836-854,...,"Keung, Emerson K. (55862195200); Rockinson-Sza...",55862195200; 35194705700,https://www.scopus.com/inward/record.uri?eid=2...,"School of Education, Liberty University, Lynch...","Keung E.K.; Rockinson-Szapkiw A.J., School of ...",Purpose: The purpose of this study is to exami...,Cultural intelligence; Intercultural schools; ...,"Alon I., Higgins J.M., Global leadership succe...",

In [11]:
## 10 ## Just in case, saving the final Scopus database in an Excel file format called "All_publications_fin_Scopus.xlsx"
#all_publications_fin.to_excel("All_publications_fin_Scopus.xlsx")

---
## Chapter_2_Data_collection_from_Scimago_Journal_Rank_(SJR)

The Scimago website does not have its own open API, so we use the "requests" package to collect data from this service. During the collection process, we found out that it would be faster and more efficient to simply download the finished data for each year by sending a request to download the finished file, since there is such a button on the website. Further selection of only the necessary journals and their ratings for each year is faster than writing a separate code to collect data for a specific journal. So, in general, we partially managed to automate the collection of data from this resource.

In [12]:
## 11 ## Downloading data on the ratings of scientific journals for each year in CSV file format

# Creating a new folder in the computer system 
# The following two lines of code only work when the code is first run, that is, if the folder has not yet been created
folder_path = "./Scimago 2013-2022 CSV"
os.makedirs(folder_path)

# The data collection loop itself
for year in range (2013, 2023):
    url = f"https://www.scimagojr.com/journalrank.php?year={year}&out=xls"
    response = requests.get(url)

    if response.status_code == 200:
        file_name = os.path.join(folder_path, f"Journal_Rankings_{year}_Scimago.csv")
        with open(file_name, "wb") as file:
            file.write(response.content)
        print(f"File saved at: {file_name}")
    else:
        print(f"Failed to retrieve the content. Status code: {response.status_code}")

File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2013_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2014_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2015_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2016_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2017_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2018_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2019_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2020_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2021_Scimago.csv
File saved at: ./Scimago 2013-2022 CSV/Journal_Rankings_2022_Scimago.csv


At the moment, the Scimago website has journals ratings only up to 2022, so data was collected for 2013-2022.

Errors occurred during further reading of the files, so we manually re-edited the downloaded CSV files into Excel format, and and saved them in a new folder "Scimago 2013-2022 XLSX".

In [13]:
## 12 ## Uploading data on journal rankings for each year
rankings_years = {}

for year in range(2013, 2023):
    file_path = f"./Scimago 2013-2022 XLSX/Journal_Rankings_{year}_Scimago.xlsx"
    rankings_years[year] = pandas.read_excel(file_path, index_col = 0)
    print(f"File for {year} opened successfully")

for year in rankings_years:
    print(f"\n--------------\nData for {year}:\n{rankings_years[year]}\n")

File for 2013 opened successfully
File for 2014 opened successfully
File for 2015 opened successfully
File for 2016 opened successfully
File for 2017 opened successfully
File for 2018 opened successfully
File for 2019 opened successfully
File for 2020 opened successfully
File for 2021 opened successfully
File for 2022 opened successfully

--------------
Data for 2013:
          Sourceid                           Journal Title     Type  \
Rank                                                                  
1            28773      Ca-A Cancer Journal for Clinicians  journal   
2            29719               Reviews of Modern Physics  journal   
3            20651             Annual Review of Immunology  journal   
4            18434                                    Cell  journal   
5            16801           Annual Review of Biochemistry  journal   
...            ...                                     ...      ...   
32588   5800212867  Zeitschrift fur Fremdsprachenforschung  j

---
## Chapter_3_Data_integration_from_Scopus_and_Scimago_Journal_Rank_(SJR)

To merge the two databases, we need to refer to the year of publication, but there is no separate variable, so we will create it based on the existing variable "coverDate" already at the current stage.

In [14]:
## 13 ## Creating new variable based on existing ones
all_publications_fin["year"] = all_publications_fin["coverDate"].apply(lambda yyyy: yyyy[:4])
all_publications_fin["year"] = all_publications_fin["year"].astype("int")
all_publications_fin[["year", "coverDate"]]

,year,coverDate
1,2022,2022-12-01
2,2022,2022-12-01
3,2022,2022-12-01
4,2022,2022-12-01
5,2022,2022-12-01
...,...,...
606,2013,2013-01-01
607,2013,2013-01-01
608,2013,2013-01-01
609,2013,2013-01-01


In [15]:
## 14 ## Merging data on publications and journals with journal ratings
all_data_raw = pandas.DataFrame()

for year in range(2013, 2023):
    rankings_data = rankings_years[year]
    one_year_data = pandas.merge(all_publications_fin[all_publications_fin["year"] == year]
                                 , rankings_data
                                 , left_on = "publicationName"
                                 , right_on= "Journal Title"
                                 , how = "left")
    
    all_data_raw = pandas.concat([all_data_raw, one_year_data])

all_data_raw = pandas.concat([all_data_raw, all_publications_fin[all_publications_fin["year"] == 2023]])    
all_data_raw

,eid,doi,subtypeDescription,affiliation_country,coverDate,publicationName,aggregationType,volume,issueIdentifier,pageRange,...,Areas,Total Docs. (2014),Total Docs. (2015),Total Docs. (2016),Total Docs. (2017),Total Docs. (2018),Total Docs. (2019),Total Docs. (2020),Total Docs. (2021),Total Docs. (2022)
0,2-s2.0-84891116423,None,Article,United States,2013-12-31,International Journal of Intangible Heritage,Journal,8,None,19-36,...,Arts and Humanities; Social Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-s2.0-84890792538,10.1002/bse.1760,Article,Taiwan,2013-12-01,Business Strategy and the Environment,Journal,22,8,517-546,...,"Business, Management and Accounting; Environme...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-s2.0-84890442458,10.1007/s10745-013-9614-8,Article,United States,2013-12-01,Human Ecology,Journal,41,6,905-914,...,Arts and Humanities; Environmental Science; Me...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-s2.0-84887148964,10.1007/s10668-013-9446-0,Article,Nigeria,2013-12-01,"Environment, Development and Sustainability",Journal,15,6,1477-1494,...,"Economics, Econometrics and Finance; Environme...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-s2.0-84887331283,None,Article,South Africa,2013-11-15,Acta Academica,Journal,45,3,63-98,...,Arts and Humanities; Social Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2-s2.0-85111312615,10.1080/09669582.2021.1953037,Article,United Kingdom,2022-01-01,Journal of Sustainable Tourism,Journal,30,6,1438-1457,...,"Business, Management and Accounting; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.0
93,2-s2.0-85105226843,10.1080/03736245.2021.1917447,Article,South Africa;South Africa,2022-01-01,South African Geographical Journal,Journal,104,1,89-106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2-s2.0-85100618204,10.1080/09669582.2021.1875477,Article,Portugal,2022-01-01,Journal of Sustainable Tourism,Journal,30,1,258-278,...,"Business, Management and Accounting; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.0
95,2-s2.0-85098564616,10.1080/15387216.2020.1867601,Article,Romania;Romania,2022-01-01,Eurasian Geography and Economics,Journal,63,3,305-331,...,"Economics, Econometrics and Finance; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0


In [16]:
## 15 ## Just in case, checking if there are any duplicates and deleting them
all_data_raw = all_data_raw.drop_duplicates("eid")
all_data_raw

,eid,doi,subtypeDescription,affiliation_country,coverDate,publicationName,aggregationType,volume,issueIdentifier,pageRange,...,Areas,Total Docs. (2014),Total Docs. (2015),Total Docs. (2016),Total Docs. (2017),Total Docs. (2018),Total Docs. (2019),Total Docs. (2020),Total Docs. (2021),Total Docs. (2022)
0,2-s2.0-84891116423,None,Article,United States,2013-12-31,International Journal of Intangible Heritage,Journal,8,None,19-36,...,Arts and Humanities; Social Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-s2.0-84890792538,10.1002/bse.1760,Article,Taiwan,2013-12-01,Business Strategy and the Environment,Journal,22,8,517-546,...,"Business, Management and Accounting; Environme...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-s2.0-84890442458,10.1007/s10745-013-9614-8,Article,United States,2013-12-01,Human Ecology,Journal,41,6,905-914,...,Arts and Humanities; Environmental Science; Me...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-s2.0-84887148964,10.1007/s10668-013-9446-0,Article,Nigeria,2013-12-01,"Environment, Development and Sustainability",Journal,15,6,1477-1494,...,"Economics, Econometrics and Finance; Environme...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-s2.0-84887331283,None,Article,South Africa,2013-11-15,Acta Academica,Journal,45,3,63-98,...,Arts and Humanities; Social Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2-s2.0-85111312615,10.1080/09669582.2021.1953037,Article,United Kingdom,2022-01-01,Journal of Sustainable Tourism,Journal,30,6,1438-1457,...,"Business, Management and Accounting; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.0
93,2-s2.0-85105226843,10.1080/03736245.2021.1917447,Article,South Africa;South Africa,2022-01-01,South African Geographical Journal,Journal,104,1,89-106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2-s2.0-85100618204,10.1080/09669582.2021.1875477,Article,Portugal,2022-01-01,Journal of Sustainable Tourism,Journal,30,1,258-278,...,"Business, Management and Accounting; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.0
95,2-s2.0-85098564616,10.1080/15387216.2020.1867601,Article,Romania;Romania,2022-01-01,Eurasian Geography and Economics,Journal,63,3,305-331,...,"Economics, Econometrics and Finance; Social Sc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0


In [17]:
## 16 ## Selecting again only the necessary characteristics from the database
characteristics = ["eid"
                   , "doi"
                   , "affiliation_country"
                   , "coverDate"
                   , "volume"
                   , "issueIdentifier"
                   , "pageRange"
                   , "citedby_count"
                   , "openaccess"
                   , "freetoreadLabel"
                   , "Authors"
                   , "Author full names"
                   , "Author(s) ID"
                   , "Affiliations"
                   , "Authors with affiliations"
                   , "Title"
                   , "Abstract"
                   , "Author Keywords"
                   , "References"
                   , "ISSN"
                   , "Link"
                   , "Direction"
                   , "Journal Title"
                   , "SJR"
                   , "SJR Best Quartile"
                   , "H index"
                   , "Categories"
                   , "Areas"]

data_raw_fin = pandas.DataFrame()

for i in all_data_raw:
    if i in characteristics:
        data_raw_fin[i] = all_data_raw[i]

data_raw_fin.index = range(1, len(data_raw_fin) + 1)
data_raw_fin

,eid,doi,affiliation_country,coverDate,volume,issueIdentifier,pageRange,citedby_count,openaccess,freetoreadLabel,...,Author Keywords,References,ISSN,Direction,Journal Title,SJR,SJR Best Quartile,H index,Categories,Areas
1,2-s2.0-84891116423,None,United States,2013-12-31,8,None,19-36,25,0,None,...,'third places'; Argentina; Bogota; Buenos Aire...,"Truscott M.C., Peopling places, storying space...",19754019,conceptual,International Journal of Intangible Heritage,0.112,Q3,9.0,Conservation (Q3); Cultural Studies (Q3); Muse...,Arts and Humanities; Social Sciences
2,2-s2.0-84890792538,10.1002/bse.1760,Taiwan,2013-12-01,22,8,517-546,7,0,None,...,Core rigidity; Economic sustainability; Enviro...,"Albino V., Balice A., Dangelico R.M., Environm...",10990836,conceptual,Business Strategy and the Environment,1.392,Q1,131.0,Business and International Management (Q1); Ge...,"Business, Management and Accounting; Environme..."
3,2-s2.0-84890442458,10.1007/s10745-013-9614-8,United States,2013-12-01,41,6,905-914,128,0,None,...,Environmental behavior measures; Pro-environme...,"Ajzen I., From intentions to actions: a theory...",03007839,both,Human Ecology,0.692,Q1,81.0,Anthropology (Q1); Sociology and Political Sci...,Arts and Humanities; Environmental Science; Me...
4,2-s2.0-84887148964,10.1007/s10668-013-9446-0,Nigeria,2013-12-01,15,6,1477-1494,85,0,None,...,Environmental attitudes; New Ecological Paradi...,"Adeola F.O., Environmental contamination, publ...",1387585X,conceptual,"Environment, Development and Sustainability",0.412,Q2,72.0,"Geography, Planning and Development (Q2); Mana...","Economics, Econometrics and Finance; Environme..."
5,2-s2.0-84887331283,None,South Africa,2013-11-15,45,3,63-98,3,0,None,...,NaN,"Anderson W.T., Cunningham W.H., The socially c...",05872405,methodological,Acta Academica,0.215,Q3,12.0,Arts and Humanities (miscellaneous) (Q3); Soci...,Arts and Humanities; Social Sciences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2-s2.0-85111312615,10.1080/09669582.2021.1953037,United Kingdom,2022-01-01,30,6,1438-1457,5,1,Green,...,activism; beach cleaning; coastal tourism; Env...,"Anderson A.R., Smith R., The moral space in en...",09669582,both,Journal of Sustainable Tourism,2.966,Q1,127.0,"Geography, Planning and Development (Q1); Tour...","Business, Management and Accounting; Social Sc..."
607,2-s2.0-85105226843,10.1080/03736245.2021.1917447,South Africa;South Africa,2022-01-01,104,1,89-106,0,0,None,...,Environmental perceptions; Gauteng; pro-enviro...,"Abubakar I., Al-Shihri F.S., Sayed M., Student...",03736245,both,NaN,NaN,NaN,NaN,NaN,NaN
608,2-s2.0-85100618204,10.1080/09669582.2021.1875477,Portugal,2022-01-01,30,1,258-278,78,0,Green,...,climate change; hospitality; Pro-environmental...,"Afsar B., Al-Ghazali B.M., Rehman Z.U., Umrani...",09669582,conceptual,Journal of Sustainable Tourism,2.966,Q1,127.0,"Geography, Planning and Development (Q1); Tour...","Business, Management and Accounting; Social Sc..."
609,2-s2.0-85098564616,10.1080/15387216.2020.1867601,Romania;Romania,2022-01-01,63,3,305-331,1,0,None,...,CEE; development politics; public enterprises;...,"Aalbers M.B., Neoliberalism Is Dead … Long Liv...",15387216,conceptual,Eurasian Geography and Economics,1.125,Q1,51.0,"Economics and Econometrics (Q1); Geography, Pl...","Economics, Econometrics and Finance; Social Sc..."


In [18]:
## 17 ## Saving the merged raw database in an Excel file format called "Data_raw_fin_Scopus+Scimago.xlsx"
data_raw_fin.index = range(1, len(data_raw_fin) + 1)
data_raw_fin.to_excel("Data_raw_fin_Scopus+Scimago.xlsx")

---
## Chapter_4_Preliminary_examination_of_the_database_and_preparation_for_analysis

### 4_1_Analysis_of_variables'_types,_missing_values_and_duplicates

In [19]:
## 18 ## Getting a general summary of the types of variables and the number of missing values in variables
data_raw_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 1 to 610
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   eid                        610 non-null    object 
 1   doi                        595 non-null    object 
 2   affiliation_country        604 non-null    object 
 3   coverDate                  610 non-null    object 
 4   volume                     602 non-null    object 
 5   issueIdentifier            525 non-null    object 
 6   pageRange                  489 non-null    object 
 7   citedby_count              610 non-null    int64  
 8   openaccess                 610 non-null    int64  
 9   freetoreadLabel            276 non-null    object 
 10  Authors                    610 non-null    object 
 11  Author full names          610 non-null    object 
 12  Author(s) ID               610 non-null    object 
 13  Link                       610 non-null    object 

In [20]:
## 19 ## Just in case, checking again for duplicate publications in the database for the "eid" variable, since there should definitely be no duplicates for this variable
data_raw_fin["eid"].duplicated().sum()

0

### 4_2_

### 4_3_

### 4_4_

---
## Chapter_5

---
## Chapter_6_Main_conclusions_on_the_results_obtained_and_the_limitations_of_the_project